# Voting

In [3]:
import matplotlib.pyplot as plt
import shap
from matplotlib import pyplot
import math
import pickle
import numpy as np
import sklearn
import sys
import pandas as pd
np.set_printoptions(threshold=sys.maxsize)


In [4]:
f_size = 13

dataset = pd.read_csv('CVD dataset-plin-0-6-7.csv') #13

data=dataset.iloc[:, :].values
X = dataset.iloc[:, :-1].values
y = dataset.iloc[:, f_size].values

rows = len(data)   
cols = len(data[0])

In [5]:
dataNoCvd = data[~(data[:,f_size] > 0.0)]
dataCvd = data[~(data[:,f_size] < 1.0)]
rowsNoCvd = len(dataNoCvd)    
rowsCvd = len(dataCvd)

In [6]:
# functions to create the appropriate limits for training, test and validation set
def find_NoCvd_limitsArray(i,DownNoCvd, UpNoCvd, fold_NoCvd_total, fold_NoCvd_residue):
    if i>0 :
        DownNoCvd = UpNoCvd
        UpNoCvd= fold_NoCvd_total+UpNoCvd
    if i < fold_NoCvd_residue :
        UpNoCvd+= 1
    return   DownNoCvd, UpNoCvd   
 
def find_Cvd_limitsArray(i,DownCvd, UpCvd, fold_Cvd_total, fold_Cvd_residue,cv):
    if i>0 :
        DownCvd = UpCvd
        UpCvd= fold_Cvd_total+UpCvd
    if i >= cv - fold_Cvd_residue :
         UpCvd+= 1     
    return   DownCvd, UpCvd  

def find_testValSubset(DownNoCvd,UpNoCvd,DownCvd,UpCvd,dataNoCvd,dataCvd):
    temp1=dataNoCvd[DownNoCvd:UpNoCvd,:]
    temp2=dataCvd[DownCvd:UpCvd,:]
    temp3=np.concatenate((temp1, temp2))
    return temp3

def find_trainSubset(DownNoCvd,UpNoCvd,DownCvd,UpCvd,dataNoCvd,dataCvd):
    temp1 = np.delete(dataNoCvd, slice(DownNoCvd, UpNoCvd), axis=0)
    temp2 = np.delete(dataCvd, slice(DownCvd, UpCvd), axis=0)
    temp3 = np.concatenate((temp1, temp2))
    return temp1, temp2, temp3

def find_trainSubset1(DownNoCvd,UpNoCvd,DownCvd,UpCvd,dataNoCvd,dataCvd):
    temp1 = np.delete(dataNoCvd, slice(DownNoCvd, UpNoCvd), axis=0)
    temp2 = np.delete(dataCvd, slice(DownCvd, UpCvd), axis=0)
    temp3 = np.delete(temp1, slice(0, 46), axis=0)
    temp4 = np.delete(temp2, slice(0, 4), axis=0)
    temp5 = np.concatenate((temp3, temp4))
    
    return temp3, temp4, temp5

In [7]:
def create_train_test_sets(f_size,cv,dataNoCvd,dataCvd,val_ratio):
    test_total = []
    train_total = []
    train_total_Cvd = []
    train_total_NoCvd = []
    validation_total = []
    
    rowsNoCvd = len(dataNoCvd)    
    rowsCvd = len(dataCvd)

    fold_Cvd_total = rowsCvd//cv
    fold_Cvd_residue= rowsCvd%cv
#     print("fold_Cvd_total  :",fold_Cvd_total," fold_Cvd_residue  :",fold_Cvd_residue)

    fold_NoCvd_total = rowsNoCvd//cv
    fold_NoCvd_residue= rowsNoCvd%cv
#     print("fold_NoCvd_total:",fold_NoCvd_total,"fold_NoCvd_residue:",fold_NoCvd_residue)
    
    Cvd_val = round(504*val_ratio*(rowsCvd/len(data)))
    noCvd_val = round(504*val_ratio)-Cvd_val
#     print(noCvd_val)


  
    DownNoCvd = 468
    UpNoCvd = 519
    DownCvd = 36
    UpCvd = 41
    

    X_test = []
    y_test = []
    X_train = []
    y_train = []
    X_val = []
    y_val = []
#     
    testSubset_total = find_testValSubset(DownNoCvd,UpNoCvd,DownCvd,UpCvd,dataNoCvd,dataCvd)        
        

    validationSubset_total = find_testValSubset(0,noCvd_val,0,Cvd_val,dataNoCvd,dataCvd)

    trainSubset_NoCvd ,trainSubset_Cvd ,trainSubset_total = find_trainSubset1(DownNoCvd,UpNoCvd,DownCvd,UpCvd,dataNoCvd,dataCvd)
    
    #creating X_train, y_train, X_test, y_test
    X_test.append(np.delete(testSubset_total, f_size, axis=1))
    y_test_temp = np.delete(testSubset_total, slice(0, f_size), axis=1)
    y_test.append(np.reshape(y_test_temp, len(y_test_temp)))
    X_test_temp = np.array(X_test)
    X_test = X_test_temp[0]
    y_test_temp = np.array(y_test)
    y_test = y_test_temp[0]
    
    X_val.append(np.delete(validationSubset_total, f_size, axis=1))
    y_val_temp = np.delete(validationSubset_total, slice(0, f_size), axis=1)
    y_val.append(np.reshape(y_val_temp, len(y_val_temp)))
    X_val_temp = np.array(X_val)
    X_val = X_val_temp[0]
    y_val_temp = np.array(y_val)
    y_val = y_val_temp[0]
        
    X_train.append(np.delete(trainSubset_total, f_size, axis=1))
    y_train_temp = np.delete(trainSubset_total, slice(0, f_size), axis=1)
    y_train.append(np.reshape(y_train_temp, len(y_train_temp)))
    X_train_temp = np.array(X_train)
    X_train = X_train_temp[0]
    y_train_temp = np.array(y_train)
    y_train = y_train_temp[0]
        

    return X_test, y_test, X_train, y_train, X_val, y_val, trainSubset_NoCvd,trainSubset_Cvd

In [8]:
X_test, y_test, X_train, y_train, X_val, y_val, trainSubset_NoCvd,trainSubset_Cvd = create_train_test_sets(f_size,10,dataNoCvd,dataCvd,0.1)

In [9]:
def predict_weighted_average(models,X):
    
    # initializing lists that we will need later
    y_pred = []
    y_pred_prob = []
    y_pred_models = []
    y_pred_models1 = []
    aver=[]
    # count the number of models
    models_size = len(models)
    
    # count the number of patients we want to predict their probability of cvd, in our case it will be always 1
    X_size = len(X)
    # initialiazing the weights
    weights = [0.2, 0.15, 0.15, 0.1, 0.15, 0.25]
    
    # y_pred_models gives the output of each model as marginal output, it will bw need for shap
    # y_pred_models1 gives the output of each model as probabilities
    for i in range(models_size):
        y_pred_models.append(models[i].predict(X,output_margin=True))  
        y_pred_models1.append(models[i].predict_proba(X)[:,1])
    
    total_shap_values = []
    total_expected_value = 0
    
    for i in range(models_size):
        # for each model we create a shap explainer
        explainer = shap.TreeExplainer(models[i],feature_perturbation = "tree_path_dependent")
        
        #  using the explainer we calculate the shapley values of each feature
        shap_values = explainer.shap_values(X)
        # we multiply the shapley values with the weight that correspond with the current model
        shap_values_weights = [[l*weights[i] for l in k] for k in shap_values]
        
        if(i==0):
             total_shap_values = shap_values_weights.copy()
        else:
            # we add the weighted shapley values of all models together
            for a in range(len(total_shap_values)):
                for b in range(len(total_shap_values[0])):
                     total_shap_values[a][b] += shap_values_weights[a][b]
        # we add together the expected value of each model
        total_expected_value += explainer.expected_value*weights[i] 
 
    t_shap_values = np.array(total_shap_values)
    
       
    for j in range(X_size):
        
        # here we perform the weighted voting
        sum = 0
        for i in range(models_size):
            sum = sum + y_pred_models[i][j]*weights[i]
        avr = sum  

        aver.append(avr)

        pred_prob = math.exp(avr)/(1 + math.exp(avr))
        y_pred_prob.append(pred_prob)
        
        if pred_prob>= 0.5:
            y_pred.append(1.)
        else:
            y_pred.append(0.)
  
            
    return y_pred,t_shap_values,y_pred_prob, total_expected_value

In [10]:
from xgboost import XGBClassifier
import pickle

def my_fit(f_size,ratio,dataNoCvd,dataCvd):   
    Subarray=[]
    models = []
    rowsNoCvd = len(dataNoCvd)    
    rowsCvd = len(dataCvd)
    
    gammas = [1.0, 1.0, 0.5, 1.0, 0.5, 1.5]
    numOfSubsamples = rowsNoCvd//(rowsCvd*ratio)
    numOfSubsamples= int(numOfSubsamples)
    SubNoCvd = rowsNoCvd//numOfSubsamples
    residue = rowsNoCvd- SubNoCvd*numOfSubsamples


    Up = 0

    # we create as many classifiers/models as occur from the partition of the data
    # in our case we will have 6
    for i in range(numOfSubsamples):
        

        
        classifier = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=0.5,
                  colsample_bynode=1, colsample_bytree=0.5, gamma=gammas[i],
                  learning_rate=0.52, max_delta_step=0, max_depth=3,
                  min_child_weight=1, missing=None, n_estimators=1000, n_jobs=1,
                  nthread=None, objective='binary:logistic', random_state=0,
                  reg_alpha=0, reg_lambda=1, scale_pos_weight=3, seed=None,
                  silent=None, subsample=0.8, verbosity=1)
        

        Down = Up
        Up= Up + SubNoCvd
        if i < residue :
            Up+= 1
        
        Sub1=dataNoCvd[Down:Up,:]
        Sub2=np.concatenate((Sub1, dataCvd))
        Subarray.append(Sub2)
        
    #Dividing to X and y of the previous traing set    
        X=np.delete(Subarray[i], f_size, axis=1)
        y=np.delete(Subarray[i], slice(0, f_size), axis=1)
        y=np.reshape(y, len(y))
        
        
       
        
        eval_set  = [(X,y), (X_val,y_val)]

        classifier.fit( X, y, eval_set=eval_set, eval_metric="auc", early_stopping_rounds=50,verbose = False)
        
        models.append(classifier)
        
        # save the model to disk
        filename = 'finalized_model.sav'
        pickle.dump(models, open(filename, 'wb'))
    return models



In [11]:
import math

def my_brier_score(y_test, y_pred_prob):
    sq_sum = 0
    for i in range(len(y_test)):
        dif = y_pred_prob[i] - y_test[i]
        sq_sum += math.pow(dif, 2)
    brier_sc = sq_sum/len(y_test)
    
    return brier_sc      

In [12]:
from sklearn.metrics import  confusion_matrix,roc_curve, roc_auc_score, accuracy_score
from sklearn import metrics
from sklearn.metrics import brier_score_loss
from statistics import mean,pstdev
import warnings
warnings.filterwarnings('ignore')

def create_ensemble_model():
  
    shap_values = []
    y_pred_total = []
    y_pred_prob_total = []
    cv_expected_value = 0

    # we training the model with the training and validation set that we created before
    models = my_fit(f_size,2,trainSubset_NoCvd,trainSubset_Cvd)
     
    y_pred,shap_values,y_pred_prob, total_expected_value = predict_weighted_average(models,X_test)
  

    # we make some ckecks with the test set that we ceated before                   
    cm = confusion_matrix(y_test, y_pred)
    #     print(cm)

    total1=sum(sum(cm))
        #####from confusion matrix calculate accuracy
    accuracy=(cm[0,0]+cm[1,1])/total1
#     print ('Accuracy : ', accuracy)

    specificity = cm[0,0]/(cm[0,0]+cm[0,1])
#     print('Specificity : ', specificity )

    sensitivity = cm[1,1]/(cm[1,0]+cm[1,1])
#     print('Sensitivity : ', sensitivity)
    brier_score = my_brier_score(y_test, y_pred_prob)
#     print(brier_score)

    auc = roc_auc_score(y_test, y_pred_prob)
#     print(auc)
 
        

    return

In [13]:
create_ensemble_model()